In [161]:
import pandas as pd

# Fuzzy Matching - 04082020

In [182]:
old=pd.read_excel(r'2015 Booth List.xlsx')
new=pd.read_excel(r'2019 Booth List.xlsx')

In [6]:
# I. Fuzzy Matching the intra localities - both old & new

In [163]:
old.head(5)

,booth_number,booth_name
0,1,Uchcha Vighalay Jamhor Purwi Bhag
1,2,Uchcha Vighalay Jamhor Purwi Madhya Bhag
2,3,Uchcha Vighalay Jamhor Pashchimi Bhag
3,4,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag
4,5,Madhya Vighalay Jamhor Purwi Bhag


In [7]:
unique_loc=new['booth_name'].unique().tolist()
type(unique_loc)

list

In [164]:
from fuzzywuzzy import process, fuzz

# LOCALITY FUZZY - edit 10082020 - (dont use, do direct match on booth first

In [8]:
# Intra fuzzy for NEW
score_sort = [(x,) + i
             for x in unique_loc 
             for i in process.extract(x, unique_loc,     scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['loc_sort','match_sort','score_sort'])
similarity_sort.head()

#Derive representative values
import numpy as np
similarity_sort['sorted_loc_sort'] = np.minimum(similarity_sort['loc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 90) &
                                      (similarity_sort['loc_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_loc_sort'] != similarity_sort['match_sort'])]

new1=new.merge(high_score_sort,how='left',left_on=new['locality'].str.upper().str.strip(),\
         right_on=high_score_sort['match_sort'].str.upper().str.strip(),indicator=True)

new1['loc_sort1']=np.where(new1['_merge']=='left_only',new1['locality'],new1['sorted_loc_sort'])

# Intra Fuzzy for OLD

unique_loc1=old['locality'].unique().tolist()
score_sort = [(x,) + i
             for x in unique_loc 
             for i in process.extract(x, unique_loc,     scorer=fuzz.token_sort_ratio)]


#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['loc_sort','match_sort','score_sort'])
similarity_sort.head()

#Derive representative values
import numpy as np
similarity_sort['sorted_loc_sort'] = np.minimum(similarity_sort['loc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 90) &
                                      (similarity_sort['loc_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_loc_sort'] != similarity_sort['match_sort'])]

old1=old.merge(high_score_sort,how='left',left_on=old['locality'].str.upper().str.strip(),\
         right_on=high_score_sort['match_sort'].str.upper().str.strip(),indicator=True)

old1['loc_sort1']=np.where(old1['_merge']=='left_only',old1['locality'],old1['sorted_loc_sort'])

In [78]:
old1[old1['loc_sort1']=='Ambedkar Aawasiya High School']

,key_0,Number,Name,locality,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
211,AMBEDKAR AAWASIYA HIGH SCHOOL,212,Ambedkar Aawasiya High School Kishanganj East ...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School
212,AMBEDKAR AAWASIYA HIGH SCHOOL,213,Ambedkar Aawasiya High School Kishanganj Middl...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School
213,AMBEDKAR AAWASIYA HIGH SCHOOL,214,Ambedkar Aawasiya High School Kishanganj West ...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School


In [80]:
old1[old1['loc_sort1']=='pahadakatata']

,key_0,Number,Name,locality,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
128,PAHADAKATTA,129,"Panchayat Bhawan Pahadakatta, East Part",pahadakatta,pahadakatata,pahadakatta,96.0,pahadakatata,both,pahadakatata
129,PAHADAKATTA,130,Panchayat Bhawan Pahadakatta West Part,pahadakatta,pahadakatata,pahadakatta,96.0,pahadakatata,both,pahadakatata


# BOOTH FUZZY - USE THIS

In [251]:
# Inter Fuzzy between new1 and old1


def checker(wrong_options,correct_options):
    
    first_pref=[]
    names_array=[]
    ratio_array=[]
    second_pref=[]
    third_pref=[]
    fourth_pref=[]
    fifth_pref=[]
    for wrong_option in wrong_options:
    
        
        
        
#         if wrong_option in correct_options:
# #             print(wrong_option)
#             names_array.append(wrong_option)
#             ratio_array.append("100")
#         else:
        x=process.extractBests(wrong_option,correct_options,scorer=fuzz.token_sort_ratio)
        first_pref.append(x[0])


        second_pref.append(x[1])
        third_pref.append(x[2])
        fourth_pref.append(x[3])
        fifth_pref.append(x[4])
    return first_pref,second_pref,third_pref,fourth_pref,fifth_pref

In [252]:
old_names=old['booth_name'].tolist()
new_names=new['booth_name'].tolist()
new_booth=new['booth_number'].tolist()

In [253]:
len(new_names),new.shape

(322, (322, 2))

In [254]:
first_pref,second_pref,third_pref,fourth_pref,fifth_pref=checker(new_names,old_names)

In [227]:
names_array

['Uchcha Vighalay Jamhor Purwi Bhag',
 'Uchcha Vighalay Jamhor Purwi Madhya Bhag',
 'Uchcha Vighalay Jamhor Pashchimi Bhag',
 'Uchcha Vighalay Jamhor Pashchimi Madhya Bhag',
 'Madhya Vighalay Jamhor Purwi Bhag',
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CAF48>,
 'Madhya Vighalay Jamhor Pashchimi Bhag',
 'Prathamik Vighalay Jokahari',
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CAED0>,
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CF048>,
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CF0C0>,
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CF138>,
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CF1B0>,
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CF228>,
 'Samudayik Bhavan Patanava',
 'Prathamik Vidyalay Devahara',
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CF2A0>,
 <generator object checker.<locals>.<genexpr> at 0x000001A2D75CF318>,
 <generator object che

In [255]:

df1 = pd.DataFrame()
df1['new_names']=pd.Series(new_names)
df1['booth_num']=pd.Series(new_booth)
# df1['perfect_match']=pd.Series(names_array)
# df1['perfect_ratio']=pd.Series(ratio_array)
df1['first_pref']=pd.Series(i[0] for i in first_pref)
df1['first_score']=pd.Series(i[1] for i in first_pref)
df1['second_pref']=pd.Series(i[0] for i in second_pref)
df1['second_score']=pd.Series(i[1] for i in second_pref)
df1['third_pref']=pd.Series(i[0] for i in third_pref)
df1['third_score']=pd.Series(i[1] for i in third_pref)
df1['fourth_pref']=pd.Series(i[0] for i in fourth_pref)
df1['fourth_score']=pd.Series(i[1] for i in fourth_pref)
df1['fifth_pref']=pd.Series(i[0] for i in fifth_pref)
df1['fifth_score']=pd.Series(i[1] for i in fifth_pref)
# df1.to_excel(‘matched_names.xlsx’, engine=’xlsxwriter’)

In [256]:
df1.shape,len(new_names)

((322, 12), 322)

In [257]:
pd.set_option('display.max_colwidth', -1)

In [178]:
df1[df1['first_score']==100].shape,df1.shape

((111, 12), (273, 12))

In [24]:
df1[df1['old_ratio1']>=90].shape,df1.shape

((242, 4), (273, 4))

In [258]:
df1.to_excel(r'matched_names_new1.xlsx',index=False)